In [ ]:
import numpy as np
import pandas as pd
import warnings
from my_analysis_toolkit import read_encrypted_excels
from my_static_data import tt_passwords, psfs_columns, psfs_basic_columns, psfs_date_columns, psfs_int_columns

In [ ]:
# Avoid representing large numbers in scientific form. To reset, use the commented line.
pd.set_option('display.float_format', '{:.1f}'.format)
# pd.reset_option('display.float_format')

# Display maximum column width:
pd.set_option('display.max_colwidth', None)

# Suppress openpyxl data validation warning
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")

In [ ]:
sheet = 'PSFS'
sp = 'pfsp'
passwords = tt_passwords
columns_names = psfs_columns
basic_columns = psfs_basic_columns
date_columns = psfs_date_columns
int_columns = psfs_int_columns

In [ ]:
tracking_tools = read_encrypted_excels(r'000', passwords) 

# Process PSFS Sheets

In [ ]:
hr = tracking_tools['tt_pt_HR_v04.xlsx'][sheet]

In [ ]:
hr

In [ ]:
hr.columns = psfs_columns
hr = hr.drop(hr.index[0]).reset_index(drop=True)
hr.dropna(subset=basic_columns, how='all', inplace=True)
hr.insert(0, sp, 'HR')

In [ ]:
hj = tracking_tools['tt_pt_HJ_v04.xlsx'][sheet]

In [ ]:
hj.columns = psfs_columns
hj = hj.drop(hr.index[0]).reset_index(drop=True)
hj.dropna(subset=basic_columns, how='all', inplace=True)
hj.insert(0, sp, 'HJ')

In [ ]:
psfs_rows = hr.shape[0] + hj.shape[0]
psfs_rows

In [ ]:
psfs_data = pd.concat([hr, hj], ignore_index=True)
psfs_data

In [ ]:
psfs_data.to_csv(r'001/1010_pt_pf_092300.csv', index=False)

# PSFS Dataframe

In [ ]:
psfs = pd.read_csv(r'001/1010_pt_pf_092300.csv')
psfs = psfs.applymap(lambda x: x.strip() if isinstance(x, str) else x)

In [ ]:
psfs.shape

In [ ]:
psfs.info()

In [ ]:
psfs

In [ ]:
psfs[int_columns] = psfs[int_columns].astype('Int64')
psfs[date_columns] = psfs[date_columns].apply(pd.to_datetime)

In [ ]:
psfs_duplicates = psfs[psfs['fcid'].duplicated(keep=False)]
psfs_duplicates

#### - HJ-pfspi: 95 = !fcid: 8110020124 | $fcid: 8110020125

#### - HJ-pfspi: 92 = !fcid: 8110110004 | $fcid: 8110100004

#### - HJ-pfspi: (88,106) = &&

#### - HR-pfspi: 19 = (!firstname, !lastname): &&

In [ ]:
psfs.loc[psfs['rid'] == 'R0142', 'fcid'] = 8110020125

In [ ]:
psfs.loc[psfs['rid'] == 'R0717', 'fcid'] = 8110100004

In [ ]:
# psfs_duplicates.to_html('psfs_duplicates.html')
psfs_duplicates.to_clipboard()

In [ ]:
psfs.to_csv(r'010/1010_pt_pf_092301.csv', index=False)

# PSFS Analysis Version

In [ ]:
pf = pd.read_csv(r'010/1010_pt_pf_092301.csv')

In [ ]:
pf

In [ ]:
pf[date_columns] = pf[date_columns].apply(pd.to_datetime)
pf[int_columns] = pf[int_columns].astype('Int64')


In [ ]:
pf.info()

In [ ]:
# Encoding binary variables
pf['sex'] = pf['sex'].replace({
    'Female': 1,
    'Male': 2
}).astype('Int64')

In [ ]:
# PSFS Analysis Complete Dataframe 
pf.to_csv(r'010/1010_pt_pf_092302.csv', index=False)

In [ ]:
# PSFS Analysis Basic Dataframe (this dataframe for analysis work - without unnecessary variables)
basic_pf = pf.copy()
basic_pf = basic_pf.drop(columns=['firstname', 'lastname', 'nat', 'sco', 'vic', 'note'])
basic_pf.to_csv(r'011/1010_pt_pf_092310.csv', index=False)

### PSFS data has a lot of NaN rows. They can't be removed before settle them with the PT service provider.

### PSFS DATA IS READY